In [2]:
library(INLA) ##
library(MASS)
library(rSPDE)
library(readr)

Loading required package: Matrix

Loading required package: sp

This is INLA_24.04.01 built 2024-04-01 19:43:22 UTC.
 - See www.r-inla.org/contact-us for how to get help.
 - List available models/likelihoods/etc with inla.list.models()
 - Use inla.doc(<NAME>) to access documentation



In [4]:
remotes::install_github("davidbolin/rspde", ref = "devel")

utf8      (1.2.3  -> 1.2.4 ) [CRAN]
withr     (2.5.0  -> 3.0.0 ) [CRAN]
fansi     (1.0.4  -> 1.0.6 ) [CRAN]
stringi   (1.7.12 -> 1.8.3 ) [CRAN]
cpp11     (0.4.6  -> 0.4.7 ) [CRAN]
vctrs     (0.6.4  -> 0.6.5 ) [CRAN]
stringr   (1.5.0  -> 1.5.1 ) [CRAN]
rlang     (1.1.1  -> 1.1.3 ) [CRAN]
lifecycle (1.0.3  -> 1.0.4 ) [CRAN]
cli       (3.6.1  -> 3.6.2 ) [CRAN]
Rcpp      (1.0.11 -> 1.0.12) [CRAN]
DBI       (1.1.3  -> 1.2.2 ) [CRAN]


Installing 12 packages: utf8, withr, fansi, stringi, cpp11, vctrs, stringr, rlang, lifecycle, cli, Rcpp, DBI

Installing packages into ‘/home/jianj0c/R/x86_64-pc-linux-gnu-library/4.3’
(as ‘lib’ is unspecified)



── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/RtmpwiXz7d/remotes1bd762d6fe7c1/davidbolin-rSPDE-df01828/DESCRIPTION’ ... OK
* preparing ‘rSPDE’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* looking to see if a ‘data/datalist’ file should be added
* building ‘rSPDE_2.3.3.9000.tar.gz’



Installing package into ‘/home/jianj0c/R/x86_64-pc-linux-gnu-library/4.3’
(as ‘lib’ is unspecified)



## generate mesh 

In [ ]:
data <- as.data.frame(read_csv("1180.SR.csv"))
logpt<-log(data$Close)#
logreturn <-logpt[2:length(logpt)]-logpt[1:(length(logpt)-1)]
date <- data$Date[1:(length(data$Date)-1)]
dates<-as.integer(format(as.Date(date),"%Y%M%d"))


## implement the model& fit model 

In [ ]:
mesh_loc <- inla.mesh.1d(dates) # 

Abar <- rspde.make.A(mesh = mesh_loc, loc = dates)

rspde_model <- rspde.matern(mesh = mesh_loc,rspde.order=1)
mesh.index <- rspde.make.index(name = "field", mesh = mesh_loc)
data = list(logreturn = logreturn, times = dates)#, field = mesh.index)
f.s <- logreturn ~   f(field, model = rspde_model) 
rspde_fit <- inla(f.s, family = "Gamma", data = data, 
        control.inla=list(int.strategy='eb'))

## Or try this, code from the inla paper

In [9]:
formula<-logreturn~f(times,model="ar1")
result=inla(formula,family="stochvol.nig",data=data)   
summary(result)

Time used:
    Pre = 0.19, Running = 1.45, Post = 0.0173, Total = 1.65 
Fixed effects:
              mean    sd 0.025quant 0.5quant 0.975quant   mode kld
(Intercept) -8.324 0.075     -8.469   -8.326      -8.17 -8.326   0

Random effects:
  Name	  Model
    times AR1 model

Model hyperparameters:
                                     mean    sd 0.025quant 0.5quant 0.975quant
skewness parameter for stochvol-nig 0.057 0.034     -0.009    0.057      0.124
shape parameter for stochvol-nig    1.015 0.016      1.002    1.010      1.056
Precision for times                 5.859 1.883      3.070    5.559     10.400
Rho for times                       0.698 0.117      0.430    0.713      0.881
                                     mode
skewness parameter for stochvol-nig 0.057
shape parameter for stochvol-nig    1.004
Precision for times                 4.995
Rho for times                       0.744

Marginal log-Likelihood:  6545.25 
 is computed 
Posterior summaries for the linear predictor and